In [1]:
import json

from pathlib import Path
from tqdm import tqdm

In [2]:
split = 'train'

all_anns = json.load(open(f'../data/vq_v2_{split}_anno.json'))
all_anns[0]

{'video_uid': '56a3b49c-6979-4253-9ff5-2733c3e2f229',
 'clip_uid': '3fe39989-54ed-4a3a-bea4-88be05ff8df7',
 'annotation_uid': 'c64628fc-f163-4eb6-b7b1-f89b35cdcf54',
 'query_set': '1',
 'clip_fps': 5.0,
 'clip_duration': 300.00000000000006,
 'original_width': 1440,
 'original_height': 1080,
 'query_frame': 1190,
 'object_title': 'mr krips',
 'visual_crop': {'fno': 1198,
  'x': 742.08,
  'y': 587.21,
  'w': 78.35,
  'h': 121.73},
 'response_track_valid_range': [1099, 1109],
 'response_track': [{'fno': 1099,
   'x': 929.68,
   'y': 540.42,
   'w': 11.2,
   'h': 41.4},
  {'fno': 1100, 'x': 917.51, 'y': 542.37, 'w': 12.18, 'h': 43.34},
  {'fno': 1101, 'x': 910.69, 'y': 546.26, 'w': 12.18, 'h': 48.12},
  {'fno': 1102, 'x': 930.17, 'y': 544.32, 'w': 18.51, 'h': 51.9},
  {'fno': 1103, 'x': 966.7, 'y': 547.33, 'w': 17.23, 'h': 56.89},
  {'fno': 1104, 'x': 1004.2, 'y': 557.32, 'w': 21.08, 'h': 62.97},
  {'fno': 1105, 'x': 1032.44, 'y': 549.99, 'w': 24.4, 'h': 74.68},
  {'fno': 1106, 'x': 1066.6

In [11]:
from PIL import Image
from decord import VideoReader
# /local_datasets/ego4d_data/v2/vq2d_frames/520ss
p_clips_dir = Path('/data/datasets/ego4d_data/v2/clips')
p_crop_out_dir = Path('../outputs/rt_pos_queries') / split

for aidx, ann in enumerate(tqdm(all_anns)):
    clip_uid = ann['clip_uid']
    qset_uuid = f"{ann['annotation_uid']}_{ann['query_set']}"
    rt = ann['response_track']
    ow, oh = ann['original_width'], ann['original_height']
    frame_idxs = [f['fno'] for f in rt]

    p_obj_dir = p_crop_out_dir / clip_uid
    p_obj_dir.mkdir(exist_ok=True, parents=True)

    p_clip = p_clips_dir / f'{clip_uid}.mp4'
    vr = VideoReader(str(p_clip))
    
    for idx, frame_idx in enumerate(frame_idxs):
        p_out = p_obj_dir / f'{clip_uid}_{frame_idx}_{qset_uuid}.jpg'
        
        if p_out.exists():
            continue
        
        frame = vr[min(6*frame_idx, len(vr)-1)].asnumpy()
        w, h = frame.shape[1], frame.shape[0]
        x1, y1, x2, y2 = rt[idx]['x'] / ow * w, rt[idx]['y'] / oh * h, (rt[idx]['x'] + rt[idx]['w']) / ow * w, (rt[idx]['y'] + rt[idx]['h']) / oh * h
        
        img = Image.fromarray(frame)
        
        if x2 - x1 == 0 or y2 - y1 == 0:
            print(f"Empty crop: {p_out}")
            cropped = img
        else:
            cropped = img.crop((x1, y1, x2, y2))
        
        cropped.save(p_out)
        if aidx % 500 == 0:
            display(cropped)

 39%|███▊      | 5264/13607 [06:33<10:22, 13.39it/s]

681.03 1105.7 681.03 1107.79


ValueError: cannot write empty image as JPEG